In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'../resources/fonts/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) # 폰트 설정
plt.rc('font', family='NanumGothic')
import seaborn as sns
import ast

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
import pandas as pd

# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
movie_path = '../datasets/movies_metadata.csv'
rating_chunks = pd.read_csv('../datasets/ratings.csv', chunksize=100000)
rating_dt = pd.concat(rating_chunks, ignore_index=True)  # 이미 데이터프레임 형태로 변환

links_path = '../datasets/links.csv'
movie_dt = pd.read_csv(movie_path)
links_dt = pd.read_csv(links_path)


In [3]:
# movie_dt에 movieId 넣어주기
movie_dt['imdb_id'] = movie_dt['imdb_id'].str.replace('tt', '', regex=False).astype(str)
links_dt['imdbId'] = links_dt['imdbId'].astype(str)

merged_movie_data = pd.merge(movie_dt, links_dt, left_on='imdb_id', right_on='imdbId', how='inner')

movie_dt['movieId'] = merged_movie_data['movieId']

# 결과 확인
print(movie_dt[['imdb_id', 'movieId', 'title']].head())
print(f"movie_dt에 추가된 movieId의 개수: {movie_dt['movieId'].nunique()}")

   imdb_id   movieId                        title
0  0114709  103208.0                    Toy Story
1  0113497  112062.0                      Jumanji
2  0113228  102174.0             Grumpier Old Men
3  0114885   53519.0            Waiting to Exhale
4  0113041   54995.0  Father of the Bride Part II
movie_dt에 추가된 movieId의 개수: 15400


In [4]:
missing_movie_ids = movie_dt['movieId'].isnull().sum()
print(f"movieId 열의 결측치 개수: {missing_movie_ids}")

movieId 열의 결측치 개수: 30057


In [5]:
movie_dt.shape

(45466, 25)

In [6]:
#결측치가 32000개 이상인건 의미없다고 판단 (영화가 4만 5천개인데 3만5천개면 )75% 결측)
drop_column_list = list(movie_dt.columns[movie_dt.isnull().sum()<=35000])
movie_dt = movie_dt[drop_column_list]

#status가 released가 아닌 영화들은 볼수 없으니 제외
#데이터셋이 과거의 자료여서 post production, In Production 등 production중인 영화들은 이미 개봉했을 수 있지만
#무비 데이터를 최신으로 변경하기 전까지는 현재 데이터셋에선 우선 제거하는걸로
movie_dt = movie_dt[movie_dt['status'] == 'Released'].dropna(subset=['status'])

#지금단계에서 필요없어보이는 column 제거
movie_dt = movie_dt.drop(['overview', 'poster_path', 'tagline', 'spoken_languages'], axis=1)

In [7]:
movie_dt['popularity'] = movie_dt['popularity'].astype(float)

In [8]:
def extract_genre_names(genres_string):
    genres_list = ast.literal_eval(genres_string)  # 문자열을 파이썬 객체로 변환
    genre_names = [genre['name'] for genre in genres_list]  # 이름만 추출
    return ', '.join(genre_names)  # 쉼표로 연결

# 장르 열 추가
movie_dt['genre_names'] = movie_dt['genres'].apply(extract_genre_names)

In [9]:
# 장르 인코딩
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

genre_encoded = mlb.fit_transform(movie_dt['genre_names'])
genre_encoded_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=movie_dt.index)
movie_dt = pd.concat([movie_dt, genre_encoded_df], axis=1)
print(movie_dt.head())


   adult    budget                                             genres     id  \
0  False  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862   
1  False  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844   
2  False         0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602   
3  False  16000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357   
4  False         0                     [{'id': 35, 'name': 'Comedy'}]  11862   

   imdb_id original_language               original_title  popularity  \
0  0114709                en                    Toy Story   21.946943   
1  0113497                en                      Jumanji   17.015539   
2  0113228                en             Grumpier Old Men   11.712900   
3  0114885                en            Waiting to Exhale    3.859495   
4  0113041                en  Father of the Bride Part II    8.387519   

                                production_companies  \
0     [{'name': 'Pixar A

In [10]:
# 개봉연도 열 추가
movie_dt['release_year'] = pd.to_datetime(movie_dt['release_date']).dt.year
movie_dt['release_year'] = movie_dt['release_year'].fillna(0).astype(int)
print(movie_dt[['title', 'release_year']].head())

                         title  release_year
0                    Toy Story          1995
1                      Jumanji          1995
2             Grumpier Old Men          1995
3            Waiting to Exhale          1995
4  Father of the Bride Part II          1995


In [11]:
# 제작사 열 추가
def extract_production_companies(companies):
    try:
        companies_list = ast.literal_eval(companies)  # 문자열을 파이썬 객체로 변환
        company_names = [company['name'] for company in companies_list]  # 이름만 추출
        return ', '.join(company_names)  # 쉼표로 연결된 문자열로 반환
    except (ValueError, SyntaxError):
        return ''

movie_dt['production_company_names'] = movie_dt['production_companies'].apply(extract_production_companies)
movie_dt['production_company_names'] = movie_dt['production_company_names'].replace('', 'Unknown').fillna('Unknown')


print(movie_dt[['title', 'production_company_names']].head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                            production_company_names  
0                            Pixar Animation Studios  
1  TriStar Pictures, Teitler Film, Interscope Com...  
2                       Warner Bros., Lancaster Gate  
3             Twentieth Century Fox Film Corporation  
4         Sandollar Productions, Touchstone Pictures  


In [12]:

# 비슷한 제작사 이름 병합
company_mapping = {
    'Warner Brothers': 'Warner Bros.',
    'Warner Bros. Pictures': 'Warner Bros.',
    'Warner Bros. Entertainment': 'Warner Bros.',
    'MGM': 'Metro-Goldwyn-Mayer (MGM)',
    'Paramount': 'Paramount Pictures',
    'Paramount Pictures Corporation': 'Paramount Pictures',
    'Twentieth Century Fox': 'Twentieth Century Fox Film Corporation',
    '20th Century Fox': 'Twentieth Century Fox Film Corporation',
    'Fox': 'Twentieth Century Fox Film Corporation',
    'Universal': 'Universal Pictures',
    'Universal Studios': 'Universal Pictures',
    'Columbia Pictures Corporation': 'Columbia Pictures',
    'Columbia': 'Columbia Pictures',
    'Canal+ Group': 'Canal+',
    'StudioCanal': 'Canal+',
    'RKO': 'RKO Radio Pictures',
    'RKO Pictures': 'RKO Radio Pictures',
    'United Artists Corporation': 'United Artists',
    'New Line': 'New Line Cinema',
    'New Line Productions': 'New Line Cinema',
    'Walt Disney Productions': 'Walt Disney Pictures',
    'Disney': 'Walt Disney Pictures',
    'Pixar Animation Studio': 'Pixar Animation Studios',
    'Pixar': 'Pixar Animation Studios',
    'Touchstone Pictures': 'Walt Disney Pictures',  # 디즈니의 계열사
    'TriStar': 'TriStar Pictures',
    'Mosfilm Cinema Concern': 'Mosfilm',
    'Miramax': 'Miramax Films',
    'France Télévisions': 'France 2 Cinéma',
    'Centre National de la Cinématographie': 'Centre National de la Cinématographie (CNC)',
    'CNC': 'Centre National de la Cinématographie (CNC)',
    'Toho': 'Toho Company',
    'BBC': 'BBC Films',
    'British Broadcasting Corporation (BBC)': 'BBC Films',
    'BBC Worldwide': 'BBC Films',
    'Gaumont Film Company': 'Gaumont',
    'Studio Canal': 'StudioCanal',
    'Relativity': 'Relativity Media',
    'TF1': 'TF1 Films Production',
    'Orion': 'Orion Pictures',
    'Village Roadshow': 'Village Roadshow Pictures',
    'Sony Pictures': 'Columbia Pictures',  # Sony 계열
    'Sony Pictures Entertainment': 'Columbia Pictures',
    # 필요에 따라 더 추가
}

movie_dt['production_company_names'] = movie_dt['production_company_names'].replace(company_mapping)

print(movie_dt[['title', 'production_company_names']].head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                            production_company_names  
0                            Pixar Animation Studios  
1  TriStar Pictures, Teitler Film, Interscope Com...  
2                       Warner Bros., Lancaster Gate  
3             Twentieth Century Fox Film Corporation  
4         Sandollar Productions, Touchstone Pictures  


In [13]:
# 상위 50개 제작사 인코딩
movie_dt['production_company_list'] = movie_dt['production_company_names'].apply(lambda x: x.split(', '))

all_companies = [company for sublist in movie_dt['production_company_list'] for company in sublist]

from collections import Counter
company_counts = Counter(all_companies)

# 상위 N개의 자주 등장하는 제작사 선택 
top_n = 52  # 'Unknown', 'The'를 제외하면 상위 50개
top_companies = [company for company, count in company_counts.most_common(top_n)]
top_companies.remove('Unknown')
top_companies.remove('The')

print(f"상위 {top_n} 제작사: {top_companies}")

movie_dt['production_company_list'] = movie_dt['production_company_names'].apply(lambda x: x.split(', '))

mlb = MultiLabelBinarizer(classes=top_companies)
encoded_companies = mlb.fit_transform(movie_dt['production_company_list'])
encoded_companies_df = pd.DataFrame(encoded_companies, columns=mlb.classes_, index=movie_dt.index)

movie_dt = pd.concat([movie_dt, encoded_companies_df], axis=1)

print(movie_dt.head())


상위 52 제작사: ['Warner Bros.', 'Metro-Goldwyn-Mayer (MGM)', 'Paramount Pictures', 'Universal Pictures', 'Twentieth Century Fox Film Corporation', 'Columbia Pictures', 'Canal+', 'Walt Disney Pictures', 'RKO Radio Pictures', 'United Artists', 'New Line Cinema', 'Columbia Pictures Corporation', 'BBC Films', 'TriStar Pictures', 'Miramax Films', 'Mosfilm', 'France 2 Cinéma', 'Centre National de la Cinématographie (CNC)', 'Toho Company', 'Touchstone Pictures', 'Gaumont', 'StudioCanal', 'Relativity Media', 'TF1 Films Production', 'Orion Pictures', 'Village Roadshow Pictures', 'Regency Enterprises', 'Universal International Pictures (UI)', 'Dimension Films', 'Zweites Deutsches Fernsehen (ZDF)', 'Lionsgate', 'DreamWorks SKG', 'Wild Bunch', 'Fox Searchlight Pictures', 'Hammer Film Productions', 'American International Pictures (AIP)', 'Svensk Filmindustri (SF)', 'Rai Cinema', 'Working Title Films', 'Hollywood Pictures', 'Channel Four Films', 'Shaw Brothers', 'Film i Väst', 'British Broadcasting Cor

In [14]:
# 상위 50개 제작사 인코딩
movie_dt['production_company_list'] = movie_dt['production_company_names'].apply(lambda x: x.split(', '))

mlb = MultiLabelBinarizer(classes=top_companies)
encoded_companies = mlb.fit_transform(movie_dt['production_company_list'])
encoded_companies_df = pd.DataFrame(encoded_companies, columns=mlb.classes_, index=movie_dt.index)

movie_dt = pd.concat([movie_dt, encoded_companies_df], axis=1)

print(movie_dt.head())

   adult    budget                                             genres     id  \
0  False  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862   
1  False  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844   
2  False         0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602   
3  False  16000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357   
4  False         0                     [{'id': 35, 'name': 'Comedy'}]  11862   

   imdb_id original_language               original_title  popularity  \
0  0114709                en                    Toy Story   21.946943   
1  0113497                en                      Jumanji   17.015539   
2  0113228                en             Grumpier Old Men   11.712900   
3  0114885                en            Waiting to Exhale    3.859495   
4  0113041                en  Father of the Bride Part II    8.387519   

                                production_companies  \
0     [{'name': 'Pixar A

In [15]:
import pandas as pd
from scipy.sparse import csr_matrix

# 데이터셋을 사용자별로 그룹화하여 시간순으로 정렬 후 분할
train_list = []
test_list = []

# 사용자별로 데이터를 시간순으로 나누기
for user_id, group in rating_dt.groupby('userId'):
    # 사용자 데이터가 최소 2개 이상 있어야 나누기 가능
    if len(group) < 2:
        continue
    
    # 시간 순서대로 정렬
    group = group.sort_values('timestamp')
    train_size = int(len(group) * 0.8)
    
    # 훈련과 테스트 데이터 분할
    train_data_part = group.iloc[:train_size]
    test_data_part = group.iloc[train_size:]
    
    # 훈련 데이터와 테스트 데이터가 비어있지 않은 경우에만 추가
    if not train_data_part.empty and not test_data_part.empty:
        train_list.append(train_data_part)
        test_list.append(test_data_part)

# 분할된 데이터를 하나의 DataFrame으로 결합
try:
    train_data = pd.concat(train_list, ignore_index=True)
except ValueError:
    print("Warning: No valid training data available, creating an empty DataFrame.")
    train_data = pd.DataFrame(columns=rating_dt.columns)

try:
    test_data = pd.concat(test_list, ignore_index=True)
except ValueError:
    print("Warning: No valid test data available, creating an empty DataFrame.")
    test_data = pd.DataFrame(columns=rating_dt.columns)

# 분할 결과 확인
print(f"훈련 데이터 크기: {train_data.shape}")
print(f"테스트 데이터 크기: {test_data.shape}")

# 사용자와 영화 ID가 훈련 데이터에 모두 포함되도록 필터링
train_user_ids = set(train_data['userId'].unique())
train_movie_ids = set(train_data['movieId'].unique())

# 테스트 데이터를 훈련 데이터에 있는 사용자와 영화로 필터링
filtered_test_data = test_data[
    test_data['userId'].isin(train_user_ids) & 
    test_data['movieId'].isin(train_movie_ids)
]

print(f"필터링된 테스트 데이터 크기: {filtered_test_data.shape}")

# CSR 형식으로 변환 (훈련 데이터가 비어있지 않은 경우에만)
if not train_data.empty:
    user_item_matrix = csr_matrix(
        (train_data['rating'], (train_data['userId'], train_data['movieId']))
    )
    print(f"CSR 행렬 크기: {user_item_matrix.shape}")
else:
    print("훈련 데이터가 비어있어 CSR 행렬을 생성할 수 없습니다.")


훈련 데이터 크기: (20714469, 4)
테스트 데이터 크기: (5304736, 4)
필터링된 테스트 데이터 크기: (5295417, 4)
CSR 행렬 크기: (270897, 176220)


In [16]:
import mlflow
import mlflow.sklearn
random_state = 123

In [17]:
mlflow.set_tracking_uri('http://localhost:5001')
display(mlflow.get_tracking_uri())

'http://localhost:5001'

In [18]:
exp = mlflow.set_experiment(experiment_name='movie_recommendation')

display(exp.name)
display(exp.experiment_id)
display(exp.artifact_location)
display(exp.creation_time)

from datetime import datetime
datetime.fromtimestamp(exp.creation_time // 1000)

'movie_recommendation'

'598438665038976004'

'mlflow-artifacts:/598438665038976004'

1728517641524

datetime.datetime(2024, 10, 10, 8, 47, 21)

In [19]:
mlflow.autolog()

2024/10/10 09:19:33 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/10/10 09:19:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [190]:
!pip install 'optuna-integration[mlflow]'

In [193]:
!pip install optuna==3.1.0 optuna-integration mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 252.5 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: optuna
    Found existing installation: optuna 4.0.0
    Uninstalling optuna-4.0.0:
      Successfully uninstalled optuna-4.0.0


In [195]:
!pip install optuna-integration mlflow --upgrade

In [27]:
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# 모델 학습과 평가 함수 정의
def train_and_evaluate(model, train_data, test_data, user_item_matrix):
    # 모델 학습
    model.fit(user_item_matrix)

    # Test 데이터셋 중 train 데이터셋에 포함된 데이터만 필터링 
    train_user_ids = set(train_data['userId'].unique())
    train_movie_ids = set(train_data['movieId'].unique())

    filtered_test_data = test_data[
        test_data['userId'].isin(train_user_ids) & 
        test_data['movieId'].isin(train_movie_ids)
    ]

    user_ids = filtered_test_data['userId'].values
    movie_ids = filtered_test_data['movieId'].values
    true_ratings = filtered_test_data['rating'].values

    # 예측된 평점 계산
    predicted_ratings = []
    for user, movie in zip(user_ids, movie_ids):
        user_vector = model.user_factors[user]
        item_vector = model.item_factors[movie]
        predicted_rating = np.dot(user_vector, item_vector)
        predicted_ratings.append(predicted_rating)

    # RMSE, MAE 계산
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    mae = mean_absolute_error(true_ratings, predicted_ratings)

    return rmse, mae

# Optuna와 MLflow 연동 설정
mlflow.set_experiment('movie_recommendation')
mlflow_callback = MLflowCallback(tracking_uri=mlflow.get_tracking_uri(), metric_name='rmse')

# Optuna 목적 함수 정의
def objective(trial):
    factors = trial.suggest_int('factors', 5, 10)
    regularization = trial.suggest_loguniform('regularization', 1e-2, 1e-1)
    iterations = trial.suggest_int('iterations', 30, 50)

    model = AlternatingLeastSquares(
        factors=factors,
        regularization=regularization,
        iterations=iterations,
        calculate_training_loss=False
    )

    rmse, mae = train_and_evaluate(model, train_data, test_data, user_item_matrix)

    # MLflow에 파라미터와 메트릭 기록
    with mlflow.start_run():
        mlflow.log_param('factors', factors)
        mlflow.log_param('regularization', regularization)
        mlflow.log_param('iterations', iterations)
        mlflow.log_metric('rmse', rmse)
        mlflow.log_metric('mae', mae)

        print(f"factors: {factors}, regularization: {regularization}, iterations: {iterations}")
        print(f"RMSE: {rmse}, MAE: {mae}")

    return rmse  # RMSE 최소화하는 방향으로 최적화

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, callbacks=[mlflow_callback])

print(f"Best hyperparameters: {study.best_params}")
print(f"Best RMSE: {study.best_value}")


[I 2024-10-10 09:48:10,143] A new study created in memory with name: no-name-f02b6248-7f90-4885-a3cd-53f7fd31bb83
100%|██████████| 38/38 [01:45<00:00,  2.78s/it]
2024/10/10 09:50:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run colorful-lamb-827 at: http://localhost:5001/#/experiments/598438665038976004/runs/c1326c1ad6ae4c6bab0921ad076c36f7.
2024/10/10 09:50:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/598438665038976004.
[I 2024-10-10 09:50:02,085] Trial 0 finished with value: 3.366323155065736 and parameters: {'factors': 5, 'regularization': 0.02423009185269481, 'iterations': 38}. Best is trial 0 with value: 3.366323155065736.
2024/10/10 09:50:02 INFO mlflow.tracking.fluent: Experiment with name 'no-name-f02b6248-7f90-4885-a3cd-53f7fd31bb83' does not exist. Creating a new experiment.
2024/10/10 09:50:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run 0 at: http://localhost:5001/#/experiments/11880905

factors: 5, regularization: 0.02423009185269481, iterations: 38
RMSE: 3.366323155065736, MAE: 3.1944228896188083


100%|██████████| 31/31 [01:37<00:00,  3.14s/it]
2024/10/10 09:51:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-auk-706 at: http://localhost:5001/#/experiments/118809058802756663/runs/14336f7e378849969edbce4e53f47721.
2024/10/10 09:51:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/118809058802756663.
[I 2024-10-10 09:51:45,395] Trial 1 finished with value: 3.3525657275017258 and parameters: {'factors': 9, 'regularization': 0.010518671699721864, 'iterations': 31}. Best is trial 1 with value: 3.3525657275017258.
2024/10/10 09:51:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run 1 at: http://localhost:5001/#/experiments/118809058802756663/runs/fb53ce8147814caf9562d949cda33fe9.
2024/10/10 09:51:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/118809058802756663.


factors: 9, regularization: 0.010518671699721864, iterations: 31
RMSE: 3.3525657275017258, MAE: 3.180764886985464


100%|██████████| 32/32 [01:25<00:00,  2.68s/it]
2024/10/10 09:53:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run chill-fish-939 at: http://localhost:5001/#/experiments/118809058802756663/runs/7fa5fb78addc4428a983ad257eb9efa0.
2024/10/10 09:53:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/118809058802756663.
[I 2024-10-10 09:53:16,486] Trial 2 finished with value: 3.3550406320000508 and parameters: {'factors': 8, 'regularization': 0.0401369490960615, 'iterations': 32}. Best is trial 1 with value: 3.3525657275017258.
2024/10/10 09:53:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run 2 at: http://localhost:5001/#/experiments/118809058802756663/runs/4e3f38659c0b4a55b2f99a8ab58ddc0f.
2024/10/10 09:53:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/118809058802756663.


...

factors: 9, regularization: 0.036993815960398926, iterations: 36
RMSE: 3.3525064523316828, MAE: 3.180710527084


100%|██████████| 33/33 [04:24<00:00,  8.02s/it]
2024/10/10 10:40:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run exultant-fly-218 at: http://localhost:5001/#/experiments/118809058802756663/runs/f2bc58522e9247f2a630ecb10b492acc.
2024/10/10 10:40:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/118809058802756663.
[I 2024-10-10 10:40:41,539] Trial 17 finished with value: 3.350874584213683 and parameters: {'factors': 10, 'regularization': 0.013274071321319381, 'iterations': 33}. Best is trial 13 with value: 3.3497934812462224.
2024/10/10 10:40:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run 17 at: http://localhost:5001/#/experiments/118809058802756663/runs/e1c3f9f539f24abea4cb80f6b99793ce.
2024/10/10 10:40:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/118809058802756663.


factors: 10, regularization: 0.013274071321319381, iterations: 33
RMSE: 3.350874584213683, MAE: 3.1791186193723626


 65%|██████▍   | 24/37 [01:15<00:41,  3.16s/it]
[W 2024-10-10 10:41:58,184] Trial 18 failed with parameters: {'factors': 9, 'regularization': 0.022003893519328764, 'iterations': 37} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/kimseohyun/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/kt/glvlq30567q0_1x9vvszbz100000gn/T/ipykernel_72687/947000040.py", line 60, in objective
    rmse, mae = train_and_evaluate(model, train_data, test_data, user_item_matrix)
  File "/var/folders/kt/glvlq30567q0_1x9vvszbz100000gn/T/ipykernel_72687/947000040.py", line 12, in train_and_evaluate
    model.fit(user_item_matrix)
  File "/Users/kimseohyun/Library/Python/3.9/lib/python/site-packages/implicit/cpu/als.py", line 163, in fit
    solver(
KeyboardInterrupt
[W 2024-10-10 10:41:58,192] Trial 18 failed with value None.


KeyboardInterrupt: 

In [ ]:
def objective_svd(trial):
    n_factors = trial.suggest_int('n_factors', 10, 50)
    regularization = trial.suggest_loguniform('regularization', 1e-4, 1e-2)
    
    model = SVD(factors=n_factors, regularization=regularization)
    model.fit(train_data)
    predictions = model.predict(val_data)
    rmse = calculate_rmse(predictions, val_labels)
    
    return rmse

# SVD 최적화 실행
study_svd = optuna.create_study(direction='minimize')
study_svd.optimize(objective_svd, n_trials=50, callbacks=[mlflow_callback])


In [21]:
!/usr/bin/python3 -m pip install optuna optuna-integration mlflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [75]:
"""from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

# 데이터셋을 사용자별로 그룹화하여 시간순으로 정렬 후 분할
train_list = []
test_list = []

# 사용자별로 데이터를 시간순으로 나누기
for user_id, group in rating_dt.groupby('userId'):
    group = group.sort_values('timestamp')
    train_size = int(len(group) * 0.8)
    train_list.append(group.iloc[:train_size])
    test_list.append(group.iloc[train_size:])

train_data = pd.concat(train_list)
test_data = pd.concat(test_list)

# 분할 결과 확인
print(f"훈련 데이터 크기: {train_data.shape}")
print(f"테스트 데이터 크기: {test_data.shape}")

# 사용자와 영화 ID가 훈련 데이터에 모두 포함되도록 필터링
train_user_ids = set(train_data['userId'].unique())
train_movie_ids = set(train_data['movieId'].unique())

# 테스트 데이터를 훈련 데이터에 있는 사용자와 영화로 필터링
filtered_test_data = test_data[
    test_data['userId'].isin(train_user_ids) & 
    test_data['movieId'].isin(train_movie_ids)
]

print(f"필터링된 테스트 데이터 크기: {filtered_test_data.shape}")

# CSR 형식으로 변환
user_item_matrix = csr_matrix((train_data['rating'], (train_data['userId'], train_data['movieId'])))"""


ValueError: No objects to concatenate

In [57]:
import sys
!{sys.executable} -m pip install optuna

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [179]:
import optuna
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Precision@K 계산 함수
def precision_at_k(recommended_items, true_items, k=10):
    recommended_items = recommended_items[:k]
    hits = len(set(recommended_items) & set(true_items))
    return hits / k

# Recall@K 계산 함수
def recall_at_k(recommended_items, true_items, k=10):
    recommended_items = recommended_items[:k]
    hits = len(set(recommended_items) & set(true_items))
    return hits / len(true_items) if len(true_items) > 0 else 0

# NDCG@K 계산 함수
def ndcg_at_k(recommended_items, true_items, k=10):
    recommended_items = recommended_items[:k]
    dcg = sum([1 / np.log2(i + 2) if recommended_items[i] in true_items else 0 for i in range(len(recommended_items))])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(len(true_items), k))])
    return dcg / idcg if idcg > 0 else 0

# 평가 함수 정의
def objective(trial):
    factors = trial.suggest_int('factors', 5, 10)
    regularization = trial.suggest_loguniform('regularization', 1e-2, 1e-1)
    iterations = trial.suggest_int('iterations', 10, 50)
    
    model = AlternatingLeastSquares(
        factors=factors,
        regularization=regularization,
        iterations=iterations
    )
    
    model.fit(user_item_matrix)
    
    # test 데이터셋 중 train 데이터셋에 포함된 데이터만 필터링 
    train_user_ids = set(train_data['userId'].unique())
    train_movie_ids = set(train_data['movieId'].unique())

    filtered_test_data = test_data[
        test_data['userId'].isin(train_user_ids) & 
        test_data['movieId'].isin(train_movie_ids)
    ]
    
    user_ids = filtered_test_data['userId'].values
    movie_ids = filtered_test_data['movieId'].values
    true_ratings = filtered_test_data['rating'].values

    # 예측된 평점 계산
    predicted_ratings = []
    for user, movie in zip(user_ids, movie_ids):
        user_vector = model.user_factors[user]
        item_vector = model.item_factors[movie]
        predicted_rating = np.dot(user_vector, item_vector)
        predicted_ratings.append(predicted_rating)

    # RMSE, MAE 계산
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    mae = mean_absolute_error(true_ratings, predicted_ratings)

    # Precision@K, Recall@K, NDCG@K 계산
    k = 10
    precision_scores = []
    recall_scores = []
    ndcg_scores = []

    for user in filtered_test_data['userId'].unique():
        user_vector = model.user_factors[user]
        scores = model.item_factors.dot(user_vector)
        recommended_items = np.argsort(scores)[::-1]  # 예측 평점이 높은 순서대로 정렬

        true_items = filtered_test_data[filtered_test_data['userId'] == user]['movieId'].tolist()

        precision = precision_at_k(recommended_items, true_items, k=k)
        recall = recall_at_k(recommended_items, true_items, k=k)
        ndcg = ndcg_at_k(recommended_items, true_items, k=k)

        precision_scores.append(precision)
        recall_scores.append(recall)
        ndcg_scores.append(ndcg)

    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_ndcg = np.mean(ndcg_scores)

    print(f"RMSE: {rmse}, MAE: {mae}, Precision@{k}: {avg_precision}, Recall@{k}: {avg_recall}, NDCG@{k}: {avg_ndcg}")

    return rmse  # 기본적으로 Optuna는 RMSE를 최소화하도록 설정, 다른 지표를 목표로 설정하려면 이 부분을 조정하세요.

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f"Best hyperparameters: {study.best_params}")
print(f"Best precisionK: {study.best_value}")


[I 2024-10-10 08:23:59,686] A new study created in memory with name: no-name-035f7d13-85b3-4415-bee7-094449580626
100%|██████████| 26/26 [01:05<00:00,  2.52s/it]
[W 2024-10-10 08:26:59,176] Trial 0 failed with parameters: {'factors': 9, 'regularization': 0.08749210015014332, 'iterations': 26} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/kimseohyun/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/kt/glvlq30567q0_1x9vvszbz100000gn/T/ipykernel_81186/682842251.py", line 76, in objective
    true_items = filtered_test_data[filtered_test_data['userId'] == user]['movieId'].tolist()
  File "/Users/kimseohyun/Library/Python/3.9/lib/python/site-packages/pandas/core/frame.py", line 4062, in __getitem__
    def __getitem__(self, key):
KeyboardInterrupt
[W 2024-10-10 08:26:59,180] Trial 0 failed with value None.


KeyboardInterrupt: 

In [30]:
# 최적의 하이퍼파라미터로 ALS 모델 재학습
best_params = study.best_params
best_model = AlternatingLeastSquares(
    factors=best_params['factors'],
    regularization=best_params['regularization'],
    iterations=best_params['iterations']
)
best_model.fit(user_item_matrix)

# 최적의 모델 성능 재평가 (전체 데이터 또는 검증 데이터 사용)
# 위에서 사용한 평가 지표들을 사용하여 다시 성능을 측정합니다.


100%|██████████| 40/40 [00:00<00:00, 69.31it/s]


In [51]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# ALS 모델 초기화
model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)
# 모델 학습
model.fit(user_item_matrix)

# RMSE 계산 함수
def calculate_rmse(true_ratings, predicted_ratings):
    return np.sqrt(mean_squared_error(true_ratings, predicted_ratings))

# MAE 계산 함수
def calculate_mae(true_ratings, predicted_ratings):
    return mean_absolute_error(true_ratings, predicted_ratings)

# --- 필터링 코드 추가 ---
# 훈련 데이터에서 사용된 사용자와 영화 ID 목록 가져오기
train_user_ids = set(train_data['userId'].unique())
train_movie_ids = set(train_data['movieId'].unique())

# 테스트 데이터를 훈련 데이터에 있는 사용자와 영화로 필터링
filtered_test_data = test_data[
    test_data['userId'].isin(train_user_ids) & 
    test_data['movieId'].isin(train_movie_ids)
]

print(f"필터링된 테스트 데이터 크기: {filtered_test_data.shape}")

# 필터링된 테스트 데이터에서 사용자와 영화 ID 추출
user_ids = filtered_test_data['userId'].values
movie_ids = filtered_test_data['movieId'].values
true_ratings = filtered_test_data['rating'].values

# ALS 모델이 예측한 평점을 저장할 리스트
predicted_ratings = []

# 사용자와 영화 ID를 통해 예측된 평점을 가져옵니다.
for user, movie in zip(user_ids, movie_ids):
    user_vector = model.user_factors[user]
    item_vector = model.item_factors[movie]
    predicted_rating = np.dot(user_vector, item_vector)
    predicted_ratings.append(predicted_rating)

# RMSE 및 MAE 계산
rmse = calculate_rmse(true_ratings, predicted_ratings)
mae = calculate_mae(true_ratings, predicted_ratings)

print(f"RMSE (콜드 스타트 처리 전): {rmse}")
print(f"MAE (콜드 스타트 처리 전): {mae}")

# 사용자별 추천 목록 생성 및 Precision@K, Recall@K, NDCG@K 계산
k = 10  # Top-K 추천에서 K 설정
precision_scores = []
recall_scores = []
ndcg_scores = []

for user in filtered_test_data['userId'].unique():
    # ALS 모델에서 해당 사용자의 영화에 대한 추천 리스트 생성
    # 추천 리스트는 해당 사용자가 아직 보지 않은 영화 중 상위 K개의 예측 평점을 기준으로 정렬
    user_vector = model.user_factors[user]
    scores = model.item_factors.dot(user_vector)
    recommended_items = np.argsort(scores)[::-1]  # 예측 평점이 높은 순서대로 정렬 (내림차순)

    # 테스트 데이터에서 해당 사용자가 실제로 평가한 영화 목록 가져오기
    true_items = filtered_test_data[filtered_test_data['userId'] == user]['movieId'].tolist()

    # Precision@K, Recall@K, NDCG@K 계산
    precision = precision_at_k(recommended_items, true_items, k=k)
    recall = recall_at_k(recommended_items, true_items, k=k)
    ndcg = ndcg_at_k(recommended_items, true_items, k=k)

    precision_scores.append(precision)
    recall_scores.append(recall)
    ndcg_scores.append(ndcg)

# 평균 계산
average_precision_at_k = np.mean(precision_scores)
average_recall_at_k = np.mean(recall_scores)
average_ndcg_at_k = np.mean(ndcg_scores)

print(f"Precision@{k} (콜드 스타트 처리 전): {average_precision_at_k}")
print(f"Recall@{k} (콜드 스타트 처리 전): {average_recall_at_k}")
print(f"NDCG@{k} (콜드 스타트 처리 전): {average_ndcg_at_k}")


100%|██████████| 20/20 [00:00<00:00, 48.25it/s]


필터링된 테스트 데이터 크기: (18732, 4)
RMSE (콜드 스타트 처리 전): 3.3364917212410434
MAE (콜드 스타트 처리 전): 3.1668033539275684
Precision@10 (콜드 스타트 처리 전): 0.03278688524590164
Recall@10 (콜드 스타트 처리 전): 0.02797384736237961
NDCG@10 (콜드 스타트 처리 전): 0.037184346223339305


In [52]:
# 유저 아이디를 입력하면 다섯 개의 추천 목록을 뽑아준다.
def recommend_movie_by_als(user_id, n_movies=5):
    # 사용자가 이미 평가한 영화와 평점 출력
    user_ratings = rating_dt[rating_dt['userId'] == user_id]
    print(f"User {user_id}의 평가 영화 및 평점:")
    for idx, row in user_ratings.iterrows():
        movie_title = movie_dt.loc[movie_dt['movieId'] == row['movieId'], 'title'].values
        if len(movie_title) > 0:
            print(f"- {movie_title[0]}: {row['rating']}점")

    # ALS 모델을 사용하여 추천 영화 목록 생성
    print(f"\nUser {user_id}에게 추천하는 영화 {n_movies}개:")
    recommendations = best_model.recommend(user_id, user_item_matrix[user_id], N=n_movies)
    for ids in recommendations[0]:
        movie_title = movie_dt.loc[movie_dt['movieId'] == ids, 'title'].values
        if len(movie_title) > 0:
            print(f"- {movie_title[0]}")

    return


In [32]:
#새 유저 정보를 넣기위한 아이디 생성기
def make_new_userId():
    return rating_dt['userId'].max()+1

In [54]:
recommend_movie_by_als(3, n_movies=5)

User 3의 평가 영화 및 평점:


KeyError: 'movieId'

In [13]:
#cold start를 막기위해, 장르별 popularity가 높은 영화를 순서대로 두개씩 뽑아, 의사를 물어보고 싶다.
#의사를 물어볼 영화 리스트를 리턴
#TODO: 여기.. 유명하긴 한데 별로 안봤을것같은 영화들이 많이 나온다. 알고리즘 바꿀 필요가 있을듯? (예: 타이타닉이 없다???)

def movielist_for_coldstart():
    result = []

    unique_genres = ['Action', 'Fantasy', 'Animation', 'Horror', 'Thriller', 'Comedy', 'Romance', 'Drama']

    for genre in unique_genres:
    # 해당 장르의 인기 영화 중에서 하나 선택
        popular_movie = movie_dt[movie_dt['genres'].str.contains(genre)].nlargest(2, 'popularity')
        result.append(popular_movie)

    result_list = pd.concat(result)
    result_list = result_list.drop_duplicates().head(10)
    return result_list

In [14]:
first_list = movielist_for_coldstart()

In [68]:
first_list.index.array

<NumpyExtensionArray>
[33356, 43644, 42222, 30700, 24455, 42902, 45202, 24351, 23675, 44271]
Length: 10, dtype: int64

In [15]:
#예를들어, 이렇게 하자.
#영화별 평점을 10개만 물어보고, 해당 평점을 토대로 추천하고싶다.

new_user_ratings = {
    'userId': make_new_userId(),  
    'movieId': first_list.index.array,
    'rating': [4, 5, 3, 4, 2, 5, 3, 4, 4, 5]  # 유저가 입력한 평점
    #평점부분 받을수있게 만들어야함
}

# DataFrame으로 변환
new_ratings_df = pd.DataFrame(new_user_ratings)

existing_movie_ids = user_item_matrix.shape[1]

# 신규 유저 평점 벡터 생성
new_user_vector = np.zeros(existing_movie_ids)

# 신규 유저의 평점을 벡터에 채워넣기
for _, row in new_ratings_df.iterrows():
    movie_index = row['movieId']  # 영화 ID
    new_user_vector[movie_index] = row['rating']  # 평점 입력

In [26]:
rating_dt['userId'].max()+1

672

In [28]:
recommend_movie_by_als(671)

['Serial Mom']
["I'm Not Rappaport"]
['Love and a .45']
["Herod's Law"]
['Bogus']


## ===========================================================================================

In [35]:
rating_table = pd.pivot_table(data=rating_dt, index='userId', columns='movieId',values='rating', fill_value= 0)

In [36]:
rating_table.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
matrix = rating_table.values
user_rating_mean = np.mean(matrix, axis=1)
matrix_user_mean = matrix - user_rating_mean.reshape(-1,1)

In [40]:
user_mean_df = pd.DataFrame(data = matrix_user_mean)
user_mean_df 

,0,1,2,3,4,5,6,7,8,9,...,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
0,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
1,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,3.970770,...,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230
2,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,...,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075
3,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,3.902162,...,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838
4,-0.043128,-0.043128,3.956872,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,...,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355,3.972645,-0.027355,-0.027355,-0.027355,-0.027355,...,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355,-0.027355
667,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,...,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273,-0.008273
668,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,...,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677,-0.013677
669,3.986984,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,...,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016,-0.013016


In [48]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(matrix_user_mean, k =12)

In [49]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(671, 12)
(12,)
(12, 9066)


In [50]:
sigma = np.diag(sigma)

In [55]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_rating_mean.reshape(-1,1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = rating_table.columns)
df_svd_preds.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.079686,0.021779,-0.013837,-0.005870,-0.028877,0.032371,0.000715,-0.004428,-0.005219,0.038195,...,-0.004324,-0.004352,0.010478,-0.004256,-0.003944,-0.005674,0.018157,-0.005575,-0.005297,-0.003766
1,1.428452,1.608841,0.529476,0.168278,0.520809,1.107473,0.529719,0.089376,0.296270,1.970031,...,0.013227,-0.002275,0.020680,-0.005245,-0.007644,-0.021019,0.031243,-0.000957,-0.000753,0.026901
2,0.977246,0.396971,0.000299,0.027444,0.021287,0.141458,-0.057134,0.031633,-0.012538,0.383576,...,0.002761,0.004907,-0.014190,-0.000251,-0.006007,-0.003189,-0.026916,0.014637,0.013287,-0.005741
3,1.870844,1.169993,0.252202,0.094831,-0.181713,-0.511953,-0.027820,-0.143080,0.013247,1.461694,...,0.026412,-0.027245,0.054681,0.018450,0.034544,-0.035740,0.088889,-0.019365,-0.017113,0.066559
4,1.182777,0.924903,0.075998,0.061505,0.602680,-0.159825,0.339925,0.081534,-0.079666,0.535018,...,-0.029124,-0.029357,0.009064,-0.029092,-0.030890,-0.057453,0.026344,-0.024027,-0.024614,-0.032752


In [56]:
def recommend_movie(df_svd_preds, user_id, ori_movie_df, ori_rating_df, num_recommendations=5):
    user_row_number = user_id-1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    user_data = ori_rating_df[ori_rating_df.userId == user_id]
    user_history = user_data.merge(ori_movie_df, on='movieId').sort_values(['rating'], ascending=False)

    recommendations = ori_movie_df[~ori_movie_df['movieId'].isin(user_history['movieId'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on='movieId')
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False)
    return user_history, recommendations

In [68]:
movie_dt['movieId'] = movie_dt.index

In [69]:
already_rated, predictions = recommend_movie(df_svd_preds, 330, movie_dt, rating_dt, 10)
already_rated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95 entries, 47 to 66
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   userId                 95 non-null     int64  
 1   movieId                95 non-null     int64  
 2   rating                 95 non-null     float64
 3   timestamp              95 non-null     int64  
 4   adult                  95 non-null     object 
 5   belongs_to_collection  13 non-null     object 
 6   budget                 95 non-null     object 
 7   genres                 95 non-null     object 
 8   homepage               5 non-null      object 
 9   id                     95 non-null     object 
 10  imdb_id                95 non-null     object 
 11  original_language      95 non-null     object 
 12  original_title         95 non-null     object 
 13  overview               93 non-null     object 
 14  popularity             95 non-null     object 
 15  poster_path 

https://lsjsj92.tistory.com/570

 ## =====================================

In [9]:
item_based_collabor = cosine_similarity(rating_table)
item_based_collabor = pd.DataFrame(data = item_based_collabor)

In [11]:
def get_item_based_collabor(movieId):
    return item_based_collabor[movieId].sort_values(ascending=False)[:6]
get_item_based_collabor(3738)

3738    1.000000
2942    0.307614
2       0.135826
2661    0.109872
6358    0.000000
6352    0.000000
Name: 3738, dtype: float64